In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
from annoy import AnnoyIndex

import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_parquet('/home/victoria/Documentos/Git projects/Two-towers/input/shopping_queries_dataset_examples.parquet')

In [21]:
data


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2621283,2621283,�����j�[�h�p�[�x abrasus,130651,B0063ASUY4,jp,E,0,1,test
2621284,2621284,�����j�[�h�p�[�x abrasus,130651,B0062EZYIG,jp,E,0,1,test
2621285,2621285,�����j�[�h�p�[�x abrasus,130651,B07H8MWBZN,jp,S,0,1,test
2621286,2621286,�����j�[�h�p�[�x abrasus,130651,B00IZH4T9S,jp,E,0,1,test


In [24]:
data_train = data[data['split'] == 'train']
data_train

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2621267,2621267,ﾚﾃﾞｨｰｽ水着,130650,B07GR94N75,jp,E,0,1,train
2621268,2621268,ﾚﾃﾞｨｰｽ水着,130650,B0769J1VB8,jp,E,0,1,train
2621269,2621269,ﾚﾃﾞｨｰｽ水着,130650,B06XD8J9MG,jp,E,0,1,train
2621270,2621270,ﾚﾃﾞｨｰｽ水着,130650,B01KZB82WQ,jp,E,0,1,train


In [25]:
data_test = data[data['split'] == 'test']
data_test

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
32,32,!qscreen fence without holes,2,B07D7TBSGH,us,I,1,1,test
33,33,!qscreen fence without holes,2,B07DHT2WZK,us,I,1,1,test
34,34,!qscreen fence without holes,2,B07DHX8YH2,us,E,1,1,test
35,35,!qscreen fence without holes,2,B07DS1YCRZ,us,S,1,1,test
36,36,!qscreen fence without holes,2,B07DS3J3MB,us,S,1,1,test
...,...,...,...,...,...,...,...,...,...
2621283,2621283,�����j�[�h�p�[�x abrasus,130651,B0063ASUY4,jp,E,0,1,test
2621284,2621284,�����j�[�h�p�[�x abrasus,130651,B0062EZYIG,jp,E,0,1,test
2621285,2621285,�����j�[�h�p�[�x abrasus,130651,B07H8MWBZN,jp,S,0,1,test
2621286,2621286,�����j�[�h�p�[�x abrasus,130651,B00IZH4T9S,jp,E,0,1,test


In [2]:
data_product = pd.read_parquet('/home/victoria/Documentos/Git projects/Two-towers/input/shopping_queries_dataset_products.parquet')[['product_id','product_title']].drop_duplicates()

: 